In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

### path 추가

In [2]:
base_path = os.path.abspath('../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [3]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [4]:
from EEM import EEM
from EDY import EDY
from EFX import EFX
from ELQ import ELQ
from EPE import EPE
from EQL import EQL
from EPM import EPM
from ESS import ESS
from EST import EST
from EVO import EVO

eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.CSV')

eindex = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
eindex1 = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
efut = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)

eindex1 = eindex1.loc[:'2008']
ERetp = eindex1.pct_change(1).iloc[1:]
ERett = eindex.pct_change(1).iloc[1:]
fRet = efut.pct_change(1).iloc[1:]

compRet = ERett
compRet[compRet.isna()] = ERetp[compRet.isna()]
ERet1 = compRet
ERet1.drop_duplicates(inplace=True)
fRet.drop_duplicates(inplace=True)
ERet = pd.concat([ERet1.loc[:'2007-12-31'], fRet.loc['2008-01-01':]], axis=0)
ERet = ERet.fillna(0)
Eindex = (1 + ERet).cumprod()
Eindex = Eindex / Eindex.iloc[0]

priceindex_mom = pd.read_csv(os.path.join(data_path, "priceindex-mom.csv"), index_col=0, parse_dates=True)
priceindex_mom.index.name = 'tdate'
priceindex_mom.columns.name = 'ticker'

dps = pd.read_csv(os.path.join(data_path, "DPS.csv"), index_col=0, parse_dates=True)
dps.index.name = 'tdate'
dps.columns.name = 'ticker'

dps1 = pd.read_csv(os.path.join(data_path, "DPS1.csv"), index_col=0, parse_dates=True)
dps1.index.name = 'tdate'
dps1.columns.name = 'ticker'

yield10 = pd.read_csv(os.path.join(data_path, "10Yield.csv"), index_col=0, parse_dates=True)
yield10.index.name = 'tdate'
yield10.columns.name = 'ticker'


# 1. EEM

- 각 전략에서 main 함수 부분을 가져오면 됩니다. 데이터가 다운이 받아져있으면 from_db = False로 하면 더 빠르게 테스트를 진행할 수 있습니다.

In [6]:
eem = EEM(strategy_name="EEM", asset_type="EQUITY")
eem.load_index_and_return(from_db=True, save_file=True) # from_db=False
eem.load_strategy_data(table='datastream', origin='ERR')
eem.set_rebalance_period(ts_freq='month', cs_freq='month')
eem.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, longlen=12, shortlen=6, lag=0)
eem.set_portfolio_parameter(cs_strategy_type='notional')
eem.make_portfolio()

2020-02-13 16:15:38,965 - EEM - INFO - [STEP 0] START LOGGING EEM
2020-02-13 16:15:38,968 - EEM - INFO - [STEP 1] LOAD DATA
2020-02-13 16:15:38,968 - EEM - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-02-13 16:15:38,990 - EEM - INFO - [STEP 1 - 3 ] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:15:44,151 - EEM - INFO - [STEP 1 - 3] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:15:48,855 - EEM - INFO - [STEP 1 - 3] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:16:32,022 - EEM - INFO - [STEP 1 - 4] SAVE RETURN DATA INTO CSV FORMAT
2020-02-13 16:16:32,865 - EEM - INFO - [STEP 2] SET REBALANCE PERIOD
2020-02-13 16:16:32,866 - EEM - INFO - [STEP 3] CACULATE SIGNAL
C:\Users\jungyoon.choi\Desktop\jungyoon\project\risk_premia\python\strategy\EEM.py:96: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/st

### PAST

In [7]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
eem_comp_df = pd.read_csv('./past/eem.csv', header=0, index_col=0, parse_dates=True)
eem_lag_comp_df = pd.read_csv('./past/eem_lag.csv', header=0, index_col=0, parse_dates=True)

- save_path를 check_path로 해주면 check 폴더에 각 전략별 성과 및 PLOT이 저장됩니다.

In [8]:
tester = Tester(eem)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

2020-02-13 16:17:34,744 - EEM - INFO - [STEP 6] START BACKTEST
2020-02-13 16:17:34,744 - EEM - INFO - [STEP 6 - 1] BACKTEST EEM TIME SERIES


EEM
0% [############################# ] 100% | ETA: 00:00:00

2020-02-13 16:17:40,183 - EEM - INFO - [STEP 6 - 2] BACKTEST EEM CROSS SECTIONAL


EEM
0% [############################# ] 100% | ETA: 00:00:00

In [9]:
to_plot = (1.+ eem_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 2. EDY

In [10]:
edy = EDY(strategy_name="EDY", asset_type="EQUITY")
edy.load_index_and_return(from_db=True, save_file=True)
edy.load_strategy_data1(table='DS', origin1='DPS', origin2='DPS1')
edy.load_strategy_data2(table='bloom', origin='10Yield')
edy.set_rebalance_period(ts_freq='month', cs_freq='month')
edy.calculate_signal(minobs1=12, nopos=0.4, CS=0.35)
edy.set_portfolio_parameter(cs_strategy_type='notional')
edy.make_portfolio()

2020-02-13 16:17:48,364 - EDY - INFO - [STEP 0] START LOGGING EDY
2020-02-13 16:17:48,365 - EDY - INFO - [STEP 1] LOAD DATA
2020-02-13 16:17:48,366 - EDY - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-02-13 16:17:48,367 - EDY - INFO - [STEP 1 - 3 ] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:17:54,406 - EDY - INFO - [STEP 1 - 3] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:17:58,887 - EDY - INFO - [STEP 1 - 3] CHANGE BLOOM DATA INTO STANDARD FORMAT
2020-02-13 16:18:36,606 - EDY - INFO - [STEP 1 - 4] SAVE RETURN DATA INTO CSV FORMAT
2020-02-13 16:18:50,427 - EDY - INFO - [STEP 2] SET REBALANCE PERIOD
C:\Users\jungyoon.choi\Desktop\jungyoon\project\risk_premia\python\strategy\EDY.py:115: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

C

### PAST

In [11]:
start_date = '2006-01-01'
end_date = '2018-05-09'

# get r results
edy_comp_df = pd.read_csv('./past/edy.csv', header=0, index_col=0, parse_dates=True)
edy_lag_comp_df = pd.read_csv('./past/edy_lag.csv', header=0, index_col=0, parse_dates=True)

In [12]:
tester = Tester(edy)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

2020-02-13 16:19:52,683 - EDY - INFO - [STEP 6] START BACKTEST
2020-02-13 16:19:52,684 - EDY - INFO - [STEP 6 - 1] BACKTEST EDY TIME SERIES


EDY
0% [############################# ] 100% | ETA: 00:00:00

2020-02-13 16:19:56,614 - EDY - INFO - [STEP 6 - 2] BACKTEST EDY CROSS SECTIONAL


EDY
0% [############################# ] 100% | ETA: 00:00:00

In [ ]:
r_data.tail(), db_data.tail()

In [ ]:
dps.tail(), edy.dps.tail()

In [ ]:
for column in dps.columns:
    (dps - edy.dps.reindex(edy.dps.index))[[column]].loc['2004':].plot(figsize=(15,10))
    plt.legend()
    plt.plot()

In [ ]:
dps1.tail(), edy.dps1.tail()

In [ ]:
for column in dps1.columns:
    (dps1 - edy.dps1.reindex(edy.dps1.index))[[column]].loc['2004':].plot(figsize=(15,10))
    plt.legend()
    plt.plot()

In [ ]:
# different column order
# different time scale csv: montly, 
yield10_adj = yield10.loc[:, edy.yield10.columns]

# for column in edy.yield10.columns:
#     (yield10_adj - edy.yield10.reindex(edy.dps.index))[[column]].loc['2004':].plot(figsize=(15,10))
#     plt.legend()
#     plt.plot()
    
for column in yield10_adj.columns:
    toplot = yield10_adj[column].to_frame()
    toplot['DB'] = edy.yield10[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')    
    

In [ ]:
yield10_adj.tail(), edy.yield10.tail()

In [ ]:
to_plot = (1.+ edy_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 3. EFX

In [ ]:
efx = EFX(strategy_name="EFX", asset_type="EQUITY")
efx.load_index_and_return(from_db=True, save_file=True)
efx.load_strategy_data(table='bloom', origin='fx')
efx.set_rebalance_period(ts_freq='month', cs_freq='month')
efx.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, longlen=12, shortlen=0, SDEV=12)
efx.set_portfolio_parameter(cs_strategy_type='notional')
efx.make_portfolio()

### PAST

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
efx_comp_df = pd.read_csv('./past/efx.csv', header=0, index_col=0, parse_dates=True)
efx_lag_comp_df = pd.read_csv('./past/efx_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(efx)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
fx = pd.read_csv(os.path.join(data_path, "fx.csv"), index_col=0, parse_dates=True)
fx.index.name = 'tdate'
fx.columns.name = 'ticker'

In [ ]:
fx.tail()

In [ ]:
efx.fx.loc['2017-12-29':, fx.columns].head()

In [ ]:
edy.dps.index

In [ ]:
for column in fx.columns:
    toplot = fx[column].to_frame()
    toplot['DB'] = efx.fx[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
to_plot = (1.+ efx_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 4. ELQ

In [ ]:
elq = ELQ(strategy_name="ELQ", asset_type="EQUITY")
elq.load_index_and_return(from_db=True, save_file=True)
elq.load_strategy_data(table='CEIC', origin='m2gdp')

elq.set_rebalance_period(ts_freq='month', cs_freq='month')
elq.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, longlen=6, shortlen=0)
elq.set_portfolio_parameter(cs_strategy_type='notional')
elq.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
elq_comp_df = pd.read_csv('./past/elq.csv', header=0, index_col=0, parse_dates=True)
elq_lag_comp_df = pd.read_csv('./past/elq_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(elq)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
m2gdp = pd.read_csv(os.path.join(data_path, "m2gdp.csv"), index_col=0, parse_dates=True)
m2gdp.index.name = 'tdate'
m2gdp.columns.name = 'ticker'
# elq.m2gdp = m2gdp

In [ ]:
m2gdp.tail(), elq.m2gdp.tail()

In [ ]:
# for column in m2gdp.columns:
#     (m2gdp - elq.m2gdp.reindex(elq.m2gdp.index))[[column]].loc['2004':].plot(figsize=(15,10))
#     plt.legend()
#     plt.plot()
    
    
for column in m2gdp.columns:
    toplot = m2gdp[column].to_frame()
    toplot['DB'] = elq.m2gdp[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
to_plot = (1.+ elq_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 5. EPE

In [ ]:
epe = EPE(strategy_name="EPE", asset_type="EQUITY")
epe.load_index_and_return(from_db=True, save_file=True)
epe.load_strategy_data(table='DS', origin1='EPS', origin2='EPS1')

epe.set_rebalance_period(ts_freq='month', cs_freq='month')
epe.calculate_signal(minobs1=12, nopos=0.4, CS=0.35)
epe.set_portfolio_parameter(cs_strategy_type='notional')
epe.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
epe_comp_df = pd.read_csv('./past/epe.csv', header=0, index_col=0, parse_dates=True)
epe_lag_comp_df = pd.read_csv('./past/epe_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(epe)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
priceindex_mom = pd.read_csv(os.path.join(data_path, "priceindex-mom.csv"), index_col=0, parse_dates=True)
priceindex_mom.index.name = 'tdate'
priceindex_mom.columns.name = 'ticker'
# epe.raw_index = priceindex_mom

eps = pd.read_csv(os.path.join(data_path, "EPS.csv"), index_col=0, parse_dates=True)
eps.index.name = 'tdate'
eps.columns.name = 'ticker'
# epe.eps = eps

eps1 = pd.read_csv(os.path.join(data_path, "EPS1.csv"), index_col=0, parse_dates=True)
eps1.index.name = 'tdate'
eps1.columns.name = 'ticker'
# epe.eps1 = eps1

In [ ]:
priceindex_mom.tail(), epe.raw_index

for column in priceindex_mom.columns:
    toplot = priceindex_mom[column].to_frame()
    toplot['DB'] = epe.raw_index[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
eps.tail(), epe.eps.tail()

for column in eps.columns:
    toplot = eps[column].to_frame()
    toplot['DB'] = epe.eps[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
eps1.tail(), epe.eps1.tail()

for column in eps1.columns:
    toplot = eps1[column].to_frame()
    toplot['DB'] = epe.eps1[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
to_plot = (1.+ epe_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 6. EPM

In [ ]:
epm = EPM(strategy_name="EPM", asset_type="EQUITY")
epm.load_index_and_return(from_db=True, save_file=True)
epm.load_strategy_data(table='datastream', origin='EPS')
epm.set_rebalance_period(ts_freq='week', cs_freq='month', rebalance_weekday=1)  # rebalance_day: monday = 0, sunday = 6
epm.calculate_signal(minobs1=52, longlen=52, longlen2=13, shortlen=2, CS=0.35)
epm.set_portfolio_parameter(cs_strategy_type='notional')
epm.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
epm_comp_df = pd.read_csv('./past/epm.csv', header=0, index_col=0, parse_dates=True)
epm_lag_comp_df = pd.read_csv('./past/epm_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(epm)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
eps = pd.read_csv(os.path.join(data_path, "EPS.csv"), index_col=0, parse_dates=True)
eps.index.name = 'tdate'
eps.columns.name = 'ticker'
# epm.eps = eps


In [ ]:
for column in eps.columns:
    toplot = eps[column].to_frame()
    toplot['DB'] = epm.eps[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
to_plot = (1.+ epm_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 7. EQL

In [ ]:
eql = EQL(strategy_name="EQL", asset_type="EQUITY")
eql.load_index_and_return(from_db=True, save_file=True)
eql.load_strategy_data(table='FS', origin1='ROA', origin2='ICR')
eql.set_rebalance_period(ts_freq='month', cs_freq='month')  # rebalance_day: monday = 0, sunday = 6
eql.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, lag=1)
eql.set_portfolio_parameter(cs_strategy_type="notional")
eql.make_portfolio()

In [ ]:
start_date = '2006-01-01'
end_date = '2018-05-09'

# get r results
eql_comp_df = pd.read_csv('./past/eql.csv', header=0, index_col=0, parse_dates=True)
eql_lag_comp_df = pd.read_csv('./past/eql_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(eql)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
roa = pd.read_csv(os.path.join(data_path, "ROA.csv"), index_col=0, parse_dates=True)
roa.index.name = 'tdate'
roa.columns.name = 'ticker'
# eql.roa = roa

icr = pd.read_csv(os.path.join(data_path, "ICR.csv"), index_col=0, parse_dates=True)
icr.index.name = 'tdate'
icr.columns.name = 'ticker'
# eql.icr = icr

In [ ]:
roa.tail(), eql.roa.tail()

for column in roa.columns:
    toplot = roa[column].to_frame()
    toplot['DB'] = eql.roa[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
icr.tail(), eql.icr.tail()


for column in icr.columns:
    toplot = icr[column].to_frame()
    toplot['DB'] = eql.icr[column]
    toplot['DIFF'] = toplot[column] - toplot['DB']
    toplot.plot(figsize=(15,10), secondary_y='DIFF')

In [ ]:
to_plot = (1.+ eql_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 8. EST

In [ ]:
est = EST(strategy_name="EST", asset_type="EQUITY")
est.load_index_and_return(from_db=True, save_file=True)
est.load_strategy_data(table='FS', origin='growthvalue')
est.set_rebalance_period(ts_freq='month', cs_freq='month')  # rebalance_day: monday = 0, sunday = 6
est.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, per=3)
est.set_portfolio_parameter(cs_strategy_type="notional")
est.make_portfolio()

In [ ]:
start_date = '2006-01-01'
end_date = '2018-05-09'

# get r results
est_comp_df = pd.read_csv('./past/est.csv', header=0, index_col=0, parse_dates=True)
est_lag_comp_df = pd.read_csv('./past/est_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(est)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
growthvalue = pd.read_csv(os.path.join(data_path, "growthvalue.csv"), index_col=0, parse_dates=True)
growthvalue.index.name = 'tdate'
growthvalue.columns.name = 'ticker'
# est.growthvalue = growthvalue

In [ ]:
growthvalue.tail(), est.growthvalue.tail()

In [ ]:
for column in growthvalue.columns:
    (growthvalue - est.growthvalue.reindex(est.growthvalue.index))[[column]].loc['2004':].plot(figsize=(15,10))
    plt.legend()
    plt.plot()

In [ ]:
to_plot = (1.+ est_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 9. EVO

In [ ]:
evo = EVO(strategy_name="EVO", asset_type="EQUITY")
evo.load_index_and_return(from_db=True, save_file=True)
evo.load_strategy_data(table='bloom', origin='ivol')
evo.set_rebalance_period(ts_freq='month', cs_freq='month')
evo.calculate_signal(minobs1=12, nopos=0.4, CS=0.35)
evo.set_portfolio_parameter(cs_strategy_type="notional")
evo.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
evo_comp_df = pd.read_csv('./past/evo.csv', header=0, index_col=0, parse_dates=True)
evo_lag_comp_df = pd.read_csv('./past/evo_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(evo)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:

ivol = pd.read_csv(os.path.join(data_path, "ivol.csv"), index_col=0, parse_dates=True)
ivol.index.name = 'tdate'
ivol.columns.name = 'ticker'
evo.ivol = ivol

In [ ]:
to_plot = (1.+ evo_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 10. ESS

In [ ]:
strategy = ESS(strategy_name="ESS", asset_type="EQUITY")
strategy.load_index_and_return(from_db=True, save_file=True)
strategy.set_rebalance_period(ts_freq='day', cs_freq = 'month')
strategy.calculate_signal(short=0.2, day1=24, CS=0.35, statwgt=1, fundwgt=1)
strategy.set_portfolio_parameter(cs_strategy_type='notional')
strategy.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
ess_comp_df = pd.read_csv('./past/ess.csv', header=0, index_col=0, parse_dates=True)
ess_lag_comp_df = pd.read_csv('./past/ess_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
# to_plot = (1.+ ess_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()